<a href="https://colab.research.google.com/github/EmmanuelECCI/tensorflow_studies/blob/main/Transferlearning_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub


from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential

In [ ]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
# Convert the images to floating-point numbers and rescale the pixel values to be between 0 and 1
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0
'''
CHATGPT suggestion 
Converting the images to floating-point numbers also allows for more precision when working with the 
pixel values. Since the weights and biases of the neural network are also represented as floating-point 
numbers, having the input data in the same format can help with the overall performance and accuracy of 
the model.
'''

'\nCHATGPT suggestion \nConverting the images to floating-point numbers also allows for more precision when working with the \npixel values. Since the weights and biases of the neural network are also represented as floating-point \nnumbers, having the input data in the same format can help with the overall performance and accuracy of \nthe model.\n'

In [ ]:

# Add a color channel dimension to the images
X_train_resized = tf.repeat(X_train[..., tf.newaxis], 3, -1)
X_test_resized = tf.repeat(X_test[..., tf.newaxis], 3, -1)


In [ ]:
X_test_resized.shape

TensorShape([10000, 28, 28, 3])

In [ ]:




# Modify the input layer to match the MNIST dataset
inputs = tf.keras.Input(shape=(28, 28, 3))
resized_inputs = tf.image.resize(inputs, (224, 224))
x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(resized_inputs)
#x = tf.keras.layers.experimental.preprocessing.Reshape((224, 224, 3))(x)

# Load the pre-trained MobileNetV2 model without the top layer
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
pretrained_model_without_top_layer = hub.KerasLayer(feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

# Connect the new input layer to the pre-trained model
x = pretrained_model_without_top_layer(x)

# Create a new model that includes the pre-trained model and the new output layer
outputs = tf.keras.layers.Dense(10, activation="softmax")(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)


In [ ]:
# Compile the model
model.summary()
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

'''
#We compile the model with the sparse_categorical_crossentropy loss function, which is appropriate for multiclass classification problems where the target values are integers
'''

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 3)]       0         
                                                                 
 tf.image.resize_1 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense_1 (Dense)             (None, 10)                12810     
                                                                 
Total params: 2,270,794
Trainable params: 12,810
Non-trainable params: 2,257,984
____________________________________________

'\n#We compile the model with the sparse_categorical_crossentropy loss function, which is appropriate for multiclass classification problems where the target values are integers\n'

In [ ]:

tb_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/",histogram_freq=1)



# Train the model
model.fit(X_train_resized, y_train, validation_data=(X_test_resized, y_test), epochs=10, batch_size=32,callbacks=[tb_callback])

Epoch 1/10
1875/1875 [==============================] - 78s 39ms/step - loss: 1.9912 - accuracy: 0.3742 - val_loss: 1.6935 - val_accuracy: 0.6237
Epoch 2/10
1875/1875 [==============================] - 75s 40ms/step - loss: 1.5258 - accuracy: 0.6382 - val_loss: 1.3758 - val_accuracy: 0.6906
Epoch 3/10
1875/1875 [==============================] - 74s 39ms/step - loss: 1.2537 - accuracy: 0.7262 - val_loss: 1.1416 - val_accuracy: 0.7786
Epoch 4/10
1875/1875 [==============================] - 74s 40ms/step - loss: 1.0708 - accuracy: 0.7755 - val_loss: 0.9706 - val_accuracy: 0.8302
Epoch 5/10
1875/1875 [==============================] - 73s 39ms/step - loss: 0.9426 - accuracy: 0.8012 - val_loss: 0.8572 - val_accuracy: 0.8299
Epoch 6/10
1875/1875 [==============================] - 73s 39ms/step - loss: 0.8438 - accuracy: 0.8231 - val_loss: 0.7929 - val_accuracy: 0.8250
Epoch 7/10
1875/1875 [==============================] - 73s 39ms/step - loss: 0.7699 - accuracy: 0.8379 - val_loss: 0.7089 -

In [ ]:

# Modify the input layer to match the MNIST dataset
inputs = tf.keras.Input(shape=(28, 28, 3))
resized_inputs = tf.image.resize(inputs, (224, 224))
x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(resized_inputs)
#x = tf.keras.layers.experimental.preprocessing.Reshape((224, 224, 3))(x)

# Load the pre-trained MobileNetV2 model without the top layer
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
pretrained_model_without_top_layer = hub.KerasLayer(feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

# Connect the new input layer to the pre-trained model
x = pretrained_model_without_top_layer(x)

# Add a hidden layer with 64 units and ReLU activation
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation="relu")(x)

# Create a new output layer with 10 units and softmax activation for classification
outputs = tf.keras.layers.Dense(10, activation="softmax")(x)

# Create the final model
model_hidden = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model_hidden.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model_hidden.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 28, 28, 3)]       0         
                                                                 
 tf.image.resize_4 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 rescaling_4 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 keras_layer_4 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense_6 (Dense)             (None, 64)                81984     
                                                           

In [ ]:
# Train the model
model_hidden.fit(X_train_resized, y_train, validation_data=(X_test_resized, y_test), epochs=10, batch_size=32,callbacks=[tb_callback])

Epoch 1/10
1875/1875 [==============================] - 77s 39ms/step - loss: 2.0218 - accuracy: 0.2848 - val_loss: 1.6451 - val_accuracy: 0.3967
Epoch 2/10
1875/1875 [==============================] - 73s 39ms/step - loss: 1.4478 - accuracy: 0.5048 - val_loss: 1.2748 - val_accuracy: 0.5338
Epoch 3/10
1875/1875 [==============================] - 73s 39ms/step - loss: 1.1870 - accuracy: 0.6052 - val_loss: 1.0619 - val_accuracy: 0.6452
Epoch 4/10
1875/1875 [==============================] - 73s 39ms/step - loss: 1.0218 - accuracy: 0.6549 - val_loss: 0.9427 - val_accuracy: 0.6766
Epoch 5/10
1875/1875 [==============================] - 73s 39ms/step - loss: 0.9304 - accuracy: 0.6818 - val_loss: 0.9231 - val_accuracy: 0.6456
Epoch 6/10
1875/1875 [==============================] - 74s 39ms/step - loss: 0.8733 - accuracy: 0.6948 - val_loss: 0.8784 - val_accuracy: 0.7001
Epoch 7/10
1875/1875 [==============================] - 74s 39ms/step - loss: 0.8344 - accuracy: 0.7094 - val_loss: 0.7946 -

In [ ]:
# Modify the input layer to match the MNIST dataset
inputs = tf.keras.Input(shape=(28, 28, 3))
resized_inputs = tf.image.resize(inputs, (224, 224))
x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(resized_inputs)
#x = tf.keras.layers.experimental.preprocessing.Reshape((224, 224, 3))(x)

# Load the pre-trained MobileNetV2 model without the top layer
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
pretrained_model_without_top_layer = hub.KerasLayer(feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

# Connect the new input layer to the pre-trained model
x = pretrained_model_without_top_layer(x)

# Add a hidden layer with 64 units and ReLU activation
x = tf.keras.layers.Dense(64, activation="relu")(x)

# Create a new output layer with 10 units and softmax activation for classification
outputs = tf.keras.layers.Dense(10, activation="softmax")(x)

# Create the final model
model_hidden_2 = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model_hidden_2.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model_hidden_2.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 28, 28, 3)]       0         
                                                                 
 tf.image.resize_5 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 rescaling_5 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 keras_layer_5 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense_8 (Dense)             (None, 64)                81984     
                                                                 
 dense_9 (Dense)             (None, 10)                650       
                                                           

In [ ]:
# Train the model
model_hidden_2.fit(X_train_resized, y_train, validation_data=(X_test_resized, y_test), epochs=10, batch_size=32,callbacks=[tb_callback])

Epoch 1/10
1875/1875 [==============================] - 78s 40ms/step - loss: 1.9404 - accuracy: 0.3334 - val_loss: 1.5103 - val_accuracy: 0.5523
Epoch 2/10
1875/1875 [==============================] - 73s 39ms/step - loss: 1.2573 - accuracy: 0.6154 - val_loss: 1.0398 - val_accuracy: 0.6531
Epoch 3/10
1875/1875 [==============================] - 73s 39ms/step - loss: 0.9370 - accuracy: 0.7103 - val_loss: 0.7987 - val_accuracy: 0.7615
Epoch 4/10
1875/1875 [==============================] - 73s 39ms/step - loss: 0.7815 - accuracy: 0.7582 - val_loss: 0.7609 - val_accuracy: 0.7592
Epoch 5/10
1875/1875 [==============================] - 73s 39ms/step - loss: 0.6799 - accuracy: 0.7903 - val_loss: 0.6228 - val_accuracy: 0.8006
Epoch 6/10
1875/1875 [==============================] - 74s 39ms/step - loss: 0.6059 - accuracy: 0.8151 - val_loss: 0.5472 - val_accuracy: 0.8307
Epoch 7/10
1875/1875 [==============================] - 74s 39ms/step - loss: 0.5525 - accuracy: 0.8300 - val_loss: 0.4778 -